In [1]:
from ortools.sat.python import cp_model
import pandas as pd

In [2]:
#df.drop(['B', 'C'], axis=1)

NameError: name 'df' is not defined

In [3]:
#Get availability
#requests = pd.read_excel('C:\\Users\\ryder\\Desktop\\ChurchesOpti\\Churchesoptimization.xlsx')
requests = requests.fillna(1)
requests = requests.drop(['Week'], axis = 1)

In [4]:
fakerequests = [[0,1,0,1], [1,1,1,1], [0,1,1,0], [1,0,0,1], [1,1,1,1], [1,0,0,1]]

In [5]:
fakerequests

[[0, 1, 0, 1],
 [1, 1, 1, 1],
 [0, 1, 1, 0],
 [1, 0, 0, 1],
 [1, 1, 1, 1],
 [1, 0, 0, 1]]

In [22]:
#Church 0 - 9 are real churches, 10-12 are dummy values to assign OPEN to
num_churches = 6
num_weeks = 4
max_assignments = 4
all_churches = range(num_churches)
all_weeks = range(num_weeks)

In [15]:
#requests.values.tolist()

In [23]:
model = cp_model.CpModel()

In [24]:
#Creates empty schedule
schedule = {}
for churches in all_churches:
    for weeks in all_weeks:
            schedule[(churches, weeks)] = model.NewBoolVar(f"church_{churches} week_{weeks}")

In [25]:
#Condition 1: Only one church can be assigned per week.
for weeks in all_weeks:
        model.AddExactlyOne(schedule[(churches, weeks)] for churches in all_churches)

In [26]:
#Condition 2: Each church can only be assigned x times per year
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])
    model.Add(sum(churchassigned) <= max_assignments)
    model.Add(sum(churchassigned) >= 0)

In [27]:
# pylint: disable=g-complex-comprehension
model.Maximize(
    sum(
        fakerequests[churches][weeks] * schedule[(churches, weeks)]
        for churches in all_churches
        for weeks in all_weeks
    )
)

In [34]:
#Can only be scheduled 1 day in a rolling 8 weeks
max_days_in_8weeks = 1

for weeks in all weeks:
    sum()    


for each 8 week window, sum of a given churches assignments = 1
    

SyntaxError: invalid syntax (3792537594.py, line 4)

In [ ]:
mingap = 2
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])
    model.Add(sum(churchassigned) <= 1)
   

In [29]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = False

In [30]:
class ChurchesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, schedule, num_churches, num_weeks, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._schedule = schedule
        self._num_churches = num_churches
        self._num_weeks = num_weeks
        #self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for weeks in range(self._num_weeks):
            print(f"Week {weeks}")
            for churches in range(self._num_churches):
                is_scheduled = False
                #for weeks in range(self._num_weeks):
                if self.Value(self._schedule[(churches, weeks)]):
                    is_scheduled = True
                    print(f"  Church {churches} is scheduled on week {weeks}")
                if not is_scheduled:
                    print(f"  Church {churches} is not scheduled")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = ChurchesPartialSolutionPrinter(
    schedule, num_churches, num_weeks, solution_limit
)

In [31]:
solver.Solve(model, solution_printer)

Solution 1
Week 0
  Church 0 is not scheduled
  Church 1 is scheduled on week 0
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 1
  Church 0 is scheduled on week 1
  Church 1 is not scheduled
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 2
  Church 0 is not scheduled
  Church 1 is scheduled on week 2
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled
Week 3
  Church 0 is scheduled on week 3
  Church 1 is not scheduled
  Church 2 is not scheduled
  Church 3 is not scheduled
  Church 4 is not scheduled
  Church 5 is not scheduled


4

In [43]:
schedule

{(0, 0): church_0 week_0(0..1),
 (0, 1): church_0 week_1(0..1),
 (0, 2): church_0 week_2(0..1),
 (0, 3): church_0 week_3(0..1),
 (0, 4): church_0 week_4(0..1),
 (0, 5): church_0 week_5(0..1),
 (0, 6): church_0 week_6(0..1),
 (0, 7): church_0 week_7(0..1),
 (0, 8): church_0 week_8(0..1),
 (0, 9): church_0 week_9(0..1),
 (0, 10): church_0 week_10(0..1),
 (0, 11): church_0 week_11(0..1),
 (0, 12): church_0 week_12(0..1),
 (0, 13): church_0 week_13(0..1),
 (0, 14): church_0 week_14(0..1),
 (0, 15): church_0 week_15(0..1),
 (0, 16): church_0 week_16(0..1),
 (0, 17): church_0 week_17(0..1),
 (0, 18): church_0 week_18(0..1),
 (0, 19): church_0 week_19(0..1),
 (0, 20): church_0 week_20(0..1),
 (0, 21): church_0 week_21(0..1),
 (0, 22): church_0 week_22(0..1),
 (0, 23): church_0 week_23(0..1),
 (0, 24): church_0 week_24(0..1),
 (0, 25): church_0 week_25(0..1),
 (0, 26): church_0 week_26(0..1),
 (0, 27): church_0 week_27(0..1),
 (0, 28): church_0 week_28(0..1),
 (0, 29): church_0 week_29(0..1),


In [16]:
model

In [33]:
print(model)

variables {
  name: "church_0 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_0 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_1 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_2 week_3"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_0"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_1"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_2"
  domain: 0
  domain: 1
}
variables {
  name: "church_3 week_3"
  

In [35]:
for church in all_churches:
    churchassigned = []
    for week in all_weeks:
        churchassigned.append(schedule[church, week])

In [36]:
churchassigned

[church_5 week_0(0..1),
 church_5 week_1(0..1),
 church_5 week_2(0..1),
 church_5 week_3(0..1)]